In [1]:
import sys

sys.path.append(
    "C:\\Users\\Wyss User\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages"
)

import biolib
import gzip
import math
import os
import re
import requests
import pathlib
import tspex

import matplotlib as mat
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests as r
import seaborn as sns
import xml.etree.ElementTree as ET

from Bio import SeqIO
from io import StringIO
from matplotlib_venn import venn2
from matplotlib_venn import venn3
from scipy.integrate import simps

In [2]:
ht_raw_data = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\LCSET-29370_AShah_MNorman_Extended_NPX_2024-02-14.parquet"
uniprot_fasta_database = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\uniprot_fasta_database.gz"
plate_layout_path = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\Plate Layout.xlsx"
assay_list_path = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\ht_panel_assay_list.xlsx"

In [27]:
ht_data = pd.read_parquet(ht_raw_data)
plate_layout = pd.read_excel(plate_layout_path)

In [81]:
data = pd.merge(ht_data, plate_layout, how="left", on="SampleID")
# repeat assays: P32455, Q02750    

In [95]:
replicate_assays = data[["UniProt"]].value_counts()
replicate_assays = replicate_assays[replicate_assays > 94]

In [98]:
replicate_assay_list = replicate_assays.index.tolist()

In [104]:
def clean_strings(strings):
    cleaned_strings = []
    for string in strings:
        cleaned_string = str(string).replace("(", "").replace(")", "").replace("'", "").replace(",", "")
        cleaned_strings.append(cleaned_string)
    return cleaned_strings

In [105]:
replicate_assay_list = clean_strings(replicate_assay_list)

In [108]:
unique_data = data[~data["UniProt"].isin(replicate_assay_list)]

In [91]:
data

,SampleID,SampleType,WellID,PlateID,DataAnalysisRefID,OlinkID,UniProt,Assay,AssayType,Panel,...,InterCV,SampleBlockQCWarn,SampleBlockQCFail,BlockQCFail,AssayQCWarn,Location,CSF_sample,Sample,Patient_ID,Health
0,SM-N8JPS,SAMPLE,A1,000011545628_A2_02-07-2024_11-47-19,D10003,OID45511,EXT1,Extension control 1,ext_ctrl,Explore_HT,...,NaN,1,1,1,0,B01,CSF2,SEC Fract 6,8376.0,Healthy
1,SM-N8JQW,SAMPLE,A2,000011545628_A2_02-07-2024_11-47-19,D10003,OID45511,EXT1,Extension control 1,ext_ctrl,Explore_HT,...,NaN,1,1,1,0,E04,CSF5,SEC Fract 9,8275.0,MCI
2,SM-N8JR1,SAMPLE,A3,000011545628_A2_02-07-2024_11-47-19,D10003,OID45511,EXT1,Extension control 1,ext_ctrl,Explore_HT,...,NaN,1,1,1,0,E08,CSF5,SEC Fract 13,8275.0,MCI
3,SM-N8JR7,SAMPLE,A4,000011545628_A2_02-07-2024_11-47-19,D10003,OID45511,EXT1,Extension control 1,ext_ctrl,Explore_HT,...,NaN,1,1,1,0,F02,CSF6,SEC Fract 7,8630.0,AD
4,SM-N8JS4,SAMPLE,A5,000011545628_A2_02-07-2024_11-47-19,D10003,OID45511,EXT1,Extension control 1,ext_ctrl,Explore_HT,...,NaN,1,1,1,0,H10,CSF8,SEC Fract 15,8781.0,MCI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511731,SM-N8JQ2,SAMPLE,H8,000011545628_A2_02-07-2024_11-47-19,D80003,OID45502,P04004,VTN,assay,Explore_HT,...,NaN,1,1,1,1,B10,CSF2,SEC Fract 15,8376.0,Healthy
511732,SM-N8JRX,SAMPLE,H9,000011545628_A2_02-07-2024_11-47-19,D80003,OID45502,P04004,VTN,assay,Explore_HT,...,NaN,1,1,1,1,H04,CSF8,SEC Fract 9,8781.0,MCI
511733,SM-N8JQJ,SAMPLE,H10,000011545628_A2_02-07-2024_11-47-19,D80003,OID45502,P04004,VTN,assay,Explore_HT,...,NaN,1,1,1,1,D03,CSF4,SEC Fract 8,8439.0,Healthy
511734,Olink_Negative_Control206,NEGATIVE_CONTROL,H11,000011545628_A2_02-07-2024_11-47-19,D80003,OID45502,P04004,VTN,assay,Explore_HT,...,NaN,0,1,1,1,NaN,NaN,NaN,NaN,NaN
